In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()
spark

In [ ]:
외부 네트워크로 접속 : 3307

In [7]:
df = spark.read.format("jdbc").options(
    url="jdbc:mysql://192.168.47.132:3307/testdb",  # VM IP 주소 사용
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="test_table",
    user="root",
    password="1234"
).load()

df.show()


+---+----+----------+
| id|name|created_at|
+---+----+----------+
+---+----+----------+



In [ ]:
# 내부 네트워크로 접근하므로 3306으로 연결

In [10]:
df = spark.read.format("jdbc").options(
    url="jdbc:mysql://mysql:3306/testdb",   # "mysql"은 docker-compose의 서비스 이름
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="test_table",
    user="root",
    password="1234"
).load()

df.show()


+---+----+----------+
| id|name|created_at|
+---+----+----------+
+---+----+----------+



In [11]:
df = spark.read.format("jdbc").options(
    url="jdbc:mysql://mysql:3306/my_db",   # "mysql"은 docker-compose의 서비스 이름
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="users",
    user="root",
    password="1234"
).load()

df.show()


+---+------+----------------+
| id|  name|           email|
+---+------+----------------+
|  1|홍길동|hong@example.com|
|  2|김철수| kim@example.com|
+---+------+----------------+



In [18]:
df = spark.read.format("jdbc").options(
    url="jdbc:mysql://mysql:3306/my_db",
    driver="com.mysql.cj.jdbc.Driver",
    query="""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'my_db'
    """,
    user="root",
    password="1234"
).load()

df.show()


+----------+
|TABLE_NAME|
+----------+
|      DEPT|
|       EMP|
|  salgrade|
|     users|
+----------+



In [ ]:
옵션 딕셔너리로 재사용

In [21]:
mysql_options = {
    "url": "jdbc:mysql://mysql:3306/my_db",
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "root",
    "password": "1234"
}

df = spark.read.format("jdbc").options(**mysql_options, dbtable="EMP").load()
df.show()


+-----+------+---------+----+----------+-------+-------+------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE|    SAL|   COMM|DEPTNO|
+-----+------+---------+----+----------+-------+-------+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800.00|   NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600.00| 300.00|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250.00| 500.00|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975.00|   NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250.00|1400.00|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850.00|   NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450.00|   NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1982-12-09|3000.00|   NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000.00|   NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500.00|   NULL|    30|
| 7876| ADAMS|    CLERK|7788|1983-01-12|1100.00|   NULL|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950.00|   NULL|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|

In [23]:
df2 = spark.read.format("jdbc").options(**mysql_options, dbtable="salgrade").load()
df2.show()

+-----+-----+-----+
|grade|losal|hisal|
+-----+-----+-----+
|    1|  700| 1200|
|    2| 1201| 1400|
|    3| 1401| 2000|
|    4| 2001| 3000|
|    5| 3001| 9999|
+-----+-----+-----+



In [24]:
spark.stop()